In [ ]:
import pandas as pd
import json
from unidecode import unidecode
from urllib.parse import unquote

In [ ]:
label_studio_file = 'ROMINA_project-6-at-2023-04-13-20-03-88a115e6.json' #json desde Label Studio --> Export --> JSON List of items in raw JSON format stored in one JSON file. Use to export both the data and the annotations for a dataset. It's Label Studio Common Format
df_full = pd.read_json(f'input/{label_studio_file}', encoding='utf-8')

In [ ]:
class Contenido_Segmento_Base(object):
    def __init__(self, text='', x=-1, y=-1, w=-1, h=-1):
        self.text = text
        self.bounding_box = {'x': x, 'y': y, 'w': w, 'h': h,}
    def to_json(self):
        return self.__dict__

class Contenido_Segmento_Nota(object):
    def __init__(self, text='', x='-1', y='-1', w='-1', h='-1'):
        self.text = text
        self.bounding_box = {'x': x, 'y': y, 'w': w, 'h': h,}
    def to_json(self):
        return self.__dict__
    
class Etiquetado(object):
    def __init__(self, diario={}, fecha={}, notas=[]):
        self.Diario = {}
        self.Fecha = {}
        self.Notas = []
        if isinstance(diario, Contenido_Segmento_Base):
            self.Diario = diario.to_json()
        if isinstance(fecha, Contenido_Segmento_Base):
            self.Fecha = fecha.to_json()
        for _ in notas:
            if isinstance(_, Etiquetado_Nota):
                self.Notas.append(_.to_json())
    def to_json(self):
        return self.__dict__

class Etiquetado_Nota(object):
    def __init__(self, volantas=[], titulos=[], copetes=[], destacados=[], cuerpos=[], fotos=[], epis=[], firmas=[], pag=[]):
        self.Volanta = []
        self.Título = []
        self.Copete = []
        self.Destacado = []
        self.Cuerpo = []
        self.Fotografía = []
        self.Epígrafe = []
        self.Firma = []
        self.Página = []
        for _ in volantas:
            if isinstance(_, Contenido_Segmento_Base):
                self.Volanta.append(_.to_json())
        for _ in titulos:
            if isinstance(_, Contenido_Segmento_Base):
                self.Título.append(_.to_json())
        for _ in copetes:
            if isinstance(_, Contenido_Segmento_Base):
                self.Copete.append(_.to_json())
        for _ in destacados:
            if isinstance(_, Contenido_Segmento_Base):
                self.Destacado.append(_.to_json())
        for _ in cuerpos:
            if isinstance(_, Contenido_Segmento_Base):
                self.Cuerpo.append(_.to_json())
        for _ in fotos:
            if isinstance(_, Contenido_Segmento_Base):
                self.Fotografía.append(_.to_json())
        for _ in epis:
            if isinstance(_, Contenido_Segmento_Base):
                self.Epígrafe.append(_.to_json())
        for _ in firmas:
            if isinstance(_, Contenido_Segmento_Base):
                self.Firma.append(_.to_json())
        for _ in pag:
            if isinstance(_, Contenido_Segmento_Base):
                self.Página.append(_.to_json())
    def to_json(self):
        return self.__dict__
    
def get_box_pixels(x=-1, y=-1, w=-1, h=-1):
    return (x, y, w, h)

def formatear_segmento(row):
    return Contenido_Segmento_Base(row.text, row.x_pixels, row.y_pixels, row.w_pixels, row.h_pixels)

In [ ]:
def get_groups(df):
    df_rectangulos = df[df['type']=='rectanglelabels']
    df_relaciones = df[df['type']=='relation']
    if df_relaciones.shape[0] == 0:
        df_rectangulos['grupo'] = 0
    else:
        df_padres = df_relaciones['from_id'].value_counts().keys()
        relaciones_id = []
        for i, p in enumerate(df_padres):
            relaciones_id.append([p] + df_relaciones[df_relaciones['from_id']==p]['to_id'].to_list())

        lista=[]
        for g in range(len(relaciones_id)):
            for r in relaciones_id[g]:
                lista.append([r,g])
        df_grupos = pd.DataFrame(lista, columns=['id', 'grupo']).set_index(['id'])

        df_rectangulos = df_rectangulos.join(df_grupos, on='id')

    return df_rectangulos

In [ ]:
segmentos_base = ['Diario', 'Fecha']
segmentos_notas = ['Volanta', 'Título', 'Copete', 'Destacado', 'Cuerpo', 'Fotografía', 'Epígrafe', 'Firma', 'Página']
segmentos_todos = segmentos_base + segmentos_notas

In [ ]:
notas = range(df_full.shape[0])
df_etiquetas = pd.DataFrame(columns=['image', 'label', 'x_pixels', 'y_pixels', 'w_pixels', 'h_pixels', 'text', 'grupo'])

index = 0

for nota in notas:
    image = df_full['data'][nota]['image']
    df_groups = get_groups(pd.DataFrame.from_dict(df_full['annotations'][nota][0]['result']))
    etiquetas = range(len(df_groups))
    for etiqueta in etiquetas:
        dict_data = {}
        dict_data['image'] = unidecode(unquote(image[24:-4]))
        dict_data['w_ori'] = df_groups.iloc[etiqueta]['original_width']
        dict_data['h_ori'] = df_groups.iloc[etiqueta]['original_height']
        dict_data['x_pixels'] = round(df_groups.iloc[etiqueta]['value']['x'] / 100 * dict_data['w_ori'])
        dict_data['y_pixels'] = round(df_groups.iloc[etiqueta]['value']['y'] / 100 * dict_data['h_ori'])
        dict_data['w_pixels'] = round(df_groups.iloc[etiqueta]['value']['width'] / 100 * dict_data['w_ori'])
        dict_data['h_pixels'] = round(df_groups.iloc[etiqueta]['value']['height'] / 100 * dict_data['h_ori'])
        dict_data['label'] = df_groups.iloc[etiqueta]['value']['rectanglelabels'][0]
        dict_data['grupo'] = df_groups.iloc[etiqueta]['grupo']
        try:
            dict_data['text'] = df_groups.iloc[etiqueta]['meta']['text'][0]
            dict_data['valido'] = True
        except:
            print(f"Segmento sin texto -> Archivo {dict_data['image']}, Etiqueta {dict_data['label']}")
            dict_data['valido'] = False
        
        df_etiquetas = pd.concat([df_etiquetas, pd.DataFrame(data=dict_data, index=[index])])
        index += 1

In [ ]:
for img in df_etiquetas.image.value_counts().keys():
    print(f'Imagen: {img}')
    yeison = {}
    nn = []
    df = df_etiquetas[(df_etiquetas.image == img)&(df_etiquetas.valido == True)]
    
    for s in segmentos_base:
        df_ = df[df.label == s]
        if df_.shape[0] > 0:
            yeison[s] = df_.apply(formatear_segmento, axis=1).to_list()
        else:
            yeison[s] = [{}]

    for g in df.grupo.value_counts().keys():
        for s in segmentos_notas:
            df_ = df[(df.label == s)&(df.grupo == g)]
            if df_.shape[0] > 0:
                yeison[s] = df_.apply(formatear_segmento, axis=1).to_list()
            else:
                yeison[s] = [{}]

        nn.append(Etiquetado_Nota(yeison['Volanta'], yeison['Título'], yeison['Copete'], yeison['Destacado'],
                                  yeison['Cuerpo'], yeison['Fotografía'], yeison['Epígrafe'], yeison['Firma'], 
                                  yeison['Página']))
    e = Etiquetado(diario=yeison['Diario'][0], fecha=yeison['Fecha'][0], notas=nn)
    
    with open(f'output/datos_{img}.json', 'w') as f:
        json.dump(e.to_json(), f, ensure_ascii=False, indent=2)
    print('Archivo generado OK!')
    print('==========================')